Copyright 2021 National Technology & Engineering Solutions<br>
of Sandia, LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS,<br>
the U.S. Government retains certain rights in this software.<br>
<br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br>
You may obtain a copy of the License at<br>
<br>
   http://www.apache.org/licenses/LICENSE-2.0<br>
<br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.

In [ ]:
import logging

In [ ]:
import numpy

In [ ]:
import cicada.additive
import cicada.communicator

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    secret_a = numpy.array(37, dtype=object)
    b = numpy.array(7, dtype=object)
    b_enc = protocol.encoder.encode(numpy.array(b, dtype=object))
    log.info(f"Player {communicator.rank} secret a: {secret_a}", src=0)
    log.info(f"Player {communicator.rank} b: {b}", src=1)
    a_share = protocol.share(src=0, secret=secret_a, shape=())#secret=protocol.encoder.encode(secret_a), shape=())
    quotient_share = protocol.division_private_public(a_share, b_enc)
    quotient = protocol.encoder.decode(protocol.reveal(quotient_share))
    log.info(f"Player {communicator.rank} quotient: {quotient}")
    actual= secret_a/b
    err_percent = 100*(quotient-actual)/actual
    log.info(f"Player {communicator.rank} quotient error %: {err_percent}", src=0)
main()